Progetto

In [6]:
import nibabel as nib
import os

# Directory di Chiara
TRAIN_DATASET_PATH = "/Users/chiaracangelosi/Documents/1-Uni/DataScience/Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"

# Percorso completo del file FLAIR
file_path = os.path.join(TRAIN_DATASET_PATH, "BraTS20_Training_355", "BraTS20_Training_355_flair.nii")

# Carica il file .nii come array NumPy
test_image_flair = nib.load(file_path).get_fdata()

# Stampa informazioni sull'immagine
print("Shape: ", test_image_flair.shape)
print("Dtype: ", test_image_flair.dtype)
print("Min: ", test_image_flair.min())
print("Max: ", test_image_flair.max())

Shape:  (240, 240, 155)
Dtype:  float64
Min:  0.0
Max:  1854.603271484375
